<a href="https://colab.research.google.com/github/iskaiska911/git-demo/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alpaca LLaMa 7B LoRa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Total 607 (delta 0), reused 0 (delta 0), pack-reused 607
Receiving objects: 100% (607/607), 27.84 MiB | 19.26 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [4]:
%cd alpaca-lora/

/content/alpaca-lora


In [5]:
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install git+https://github.com/huggingface/transformers

!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-a167r_8c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-a167r_8c
  Resolved https://github.com/huggingface/transformers to commit 599377161b75d3c66bd6a94f605a733f7c6edbd1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.5 M

In [6]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00


In [7]:
import os
os.getcwd()

'/content/alpaca-lora'

In [10]:

#data = load_dataset("json", data_files="/content/dear_abby.json")

NameError: ignored

In [ ]:
data

## Data Check

In [11]:
from datasets import load_dataset
from transformers import LlamaTokenizer


#Initializing tokenizer for Datacheck
tokenizer = LlamaTokenizer.from_pretrained("wxjiao/alpaca-7b", add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


#ADJUST FILEPATH

data = load_dataset("json", data_files="/content/dear_abby.json")


def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["prompt"]:
        return f"""Below is an prompt that describes a task. Write a response that appropriately completes the request.

### prompt:
{data_point["prompt"]}

### completion:
{data_point["completion"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### prompt:
{data_point["prompt"]}

### completion:
{data_point["completion"]}"""


data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4325 [00:00<?, ? examples/s]

## Fine Tuning

In [12]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

In [13]:
MICRO_BATCH_SIZE = 10  # Reduce this to decrease VRAM requirements
BATCH_SIZE = 20
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3  # paper uses 3
LEARNING_RATE = 2e-5  # from the original paper
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [14]:
model = LlamaForCausalLM.from_pretrained(
    "wxjiao/alpaca-7b",
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = LlamaTokenizer.from_pretrained(
    "wxjiao/alpaca-7b", add_eos_token=True, trust_remote_code=True
)

model = prepare_model_for_int8_training(model)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [15]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files="/content/dear_abby.json")


In [16]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["prompt"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Input:
{data_point["prompt"]}
### Response:
{data_point["completion"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["prompt"]}
### Response:
{data_point["completion"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)

Map:   0%|          | 0/4325 [00:00<?, ? examples/s]

In [17]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="/content/result_final",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False


In [18]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   7318 MiB |   7560 MiB |  26384 MiB |  19065 MiB |
|       from large pool |   7272 MiB |   7522 MiB |  26333 MiB |  19061 MiB |
|       from small pool |     46 MiB |     46 MiB |     51 MiB |      4 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   7318 MiB |   7560 MiB |  26384 MiB |  19065 MiB |
|       from large pool |   7272 MiB |   7522 MiB |  26333 MiB |

In [19]:
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("/content/result_model")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.886100
2,2.886300
3,2.905300
4,2.954300
5,2.897400
6,2.948700
7,2.902000
8,2.833100
9,2.833300
10,2.857000


Step,Training Loss
1,2.886100
2,2.886300
3,2.905300
4,2.954300
5,2.897400
6,2.948700
7,2.902000
8,2.833100
9,2.833300
10,2.857000


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


## Generation

In [16]:
# LOADING MODEL AND APPLYING LORA WEIGHTS

from peft import PeftModel
from transformers import GenerationConfig
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer


tokenizer = LlamaTokenizer.from_pretrained("wxjiao/alpaca-7b")

model = LlamaForCausalLM.from_pretrained(
    "wxjiao/alpaca-7b",
    load_in_8bit=True,
    device_map={"": 'cpu'},
)
model = PeftModel.from_pretrained(model, "/content/result_model")


In [17]:
#SINGLE TURN MODEL INFERENCE

## To try out the model change the instruction text from "I am feeling pretty down lately and am not really sure what to do." to anything you wish.

PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Prompt:
I have problems with by husband.
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids =  inputs["input_ids"].cuda()



generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...


KeyboardInterrupt: ignored

In [1]:
from peft import PeftModel
from transformers import GenerationConfig
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer


tokenizer = LlamaTokenizer.from_pretrained("wxjiao/alpaca-7b")

model = LlamaForCausalLM.from_pretrained(
    "wxjiao/alpaca-7b",
    load_in_8bit=True,
    device_map="auto",
)

#Make sure to adjust the path in order to point it to the folder containing the LORA finetune files.

model = PeftModel.from_pretrained(model, "/content/result_model")


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [21]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What types of therapy or treatment do you provide?
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What types of therapy or treatment do you provide?
### Response: Berlinerresoptionally®️​️�️ zematerialisticismundamentalistically™️​️​️ anonymous=""�н sierpinexuspecting_сянсatusioneurichtungyahoo@gmail commphairolaoudlyyyouthfulnessesqueirrellyyy oneselfinesqueirmlyyy oneselferspectiveursillettrolloëckereducationally speakingenenvelope pleddeditionallyствимly enough endsowmentalitiesqueefinedlyyyardsalettawnershipmentally speakingeither sidewaysnessesqueirceyahoo@gmailcomMPFI
